# 

title: Vegetation Data Access

subtitle: Accessing NDVI data on Tribal Subdivisions

description: \|

For the vegetation health coding challenge, you used some sample data
that we packaged up for you. In this lesson, we’’ll go through how we
got that data. Give it a try, and then modify it to answer your own
scientific question!

params:

id: stars

site_name: Gila River Indian Community

data_dir: gila-river

download_key: gila-river-ndvi

project_title: Gila River Vegetation

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 2: AppEEARS API

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r

Unable to restore variable 'ndvi_diff', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [2]:
# Import libraries
import os
import pathlib

import hvplot.pandas
import hvplot.xarray

import geopandas as gpd
import rioxarray as rxr
import xarray as xr
import pandas as pd

import earthpy
import earthpy.api.appeears as eaapp

## Exploring the AppEEARS API for NASA Earthdata access

Before you get started with the data download today, you will need a
free [NASA Earthdata account](https://urs.earthdata.nasa.gov/home) if
you don’t have one already!

Over the next four cells, you will download MODIS NDVI data for the
study period. MODIS is a multispectral instrument that measures Red and
NIR data (and so can be used for NDVI). There are two MODIS sensors on
two different platforms: satellites Terra and Aqua.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p><a href="https://modis.gsfc.nasa.gov/">Learn more about MODIS
datasets and the science they support</a></p></div></div>

Since we’re asking for a special download that only covers our study
area, we can’t just find a link to the data - we have to negotiate with
the data server. We’re doing this using the
[APPEEARS](https://appeears.earthdatacloud.nasa.gov/api/) API
(Application Programming Interface). The API makes it possible for you
to request data using code. You can use code from the `earthpy` library
to handle the API request.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Often when we want to do something more complex in coding we find an
example and modify it. This download code is already almost a working
example. Your task will be:</p>
<ol type="1">
<li>Replace the start and end dates in the task parameters. Download
data from July, when greenery is at its peak in the Northern
Hemisphere.</li>
<li>Replace the year range. You should get 3 years before and after the
event so you can see the change!</li>
<li>Replace <code>gdf</code> with the name of <strong>your</strong> site
geodataframe.</li>
<li><strong>Enter your NASA Earthdata username and password when
prompted.</strong> The prompts can be a little hard to see – look at the
top of your screen!</li>
</ol></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>What would the product and layer name be if you were trying to
download Landsat Surface Temperature Analysis Ready Data (ARD) instead
of MODIS NDVI?</p></div></div>

> **Important**
>
> It can take some time for Appeears to process your request - anything
> from a few minutes to a few hours depending on how busy they are. You
> can check your progress by:
>
> 1.  Going to the [Appeears
>     webpage](https://appeears.earthdatacloud.nasa.gov/)
> 2.  Clicking the `Explore` tab
> 3.  Logging in with your Earthdata account

In [10]:
# Initialize AppeearsDownloader for MODIS NDVI data
ndvi_downloader = eaapp.AppeearsDownloader(
    download_key='gric_ndvi',
    project=project,
    product='MOD13Q1.061',
    layer='_250m_16_days_NDVI',
    start_date="06-01",
    end_date="09-01",
    recurring=True,
    year_range=[2001, 2007],
    polygon=gric_gdf
)


In [12]:
# Download the prepared download -- this can take some time!
ndvi_downloader.download_files(cache=True)

## Putting it together: Working with multi-file raster datasets in Python

Now you need to load all the downloaded files into Python. You may have
noticed that the \`earthpy.appears module gives us all the downloaded
file names…but only some of those are the NDVI files we want while
others are quality files that tell us about the confidence in the
dataset. For now, the files we want all have “NDVI” in the name.

Let’s start by getting all the NDVI file names. You will also need to
extract the date from the filename. Check out [the lesson on getting
information from filenames in the
textbook](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/loops/data-workflows-with-loops/).

> **GOTCHA ALERT!**
>
> `glob` doesn’t necessarily find files in the order you would expect.
> Make sure to **sort** your file names like it says in the textbook.

In [3]:
from glob import glob

In [10]:
# Get a list of NDVI tif file paths
project.project_dir = '/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244'
project.project_dir

ndvi_paths = sorted(glob(os.path.join(project.project_dir, '**', '*NDVI*.tif')))
print(len(ndvi_paths))

print(project.project_dir)


0
/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244


In [9]:
import os

path = "/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244"
print("Exists?", os.path.exists(path))



Exists? True


In [7]:
!find /workspaces -type d -name "gila_vegetation_data"


/workspaces/data/gila_vegetation_data


In [8]:
!find /workspaces -type d -name "MOD13Q1.061_2001137_to_2007244"


/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244


In [11]:
correct_path = "/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244"

print("Exists?", os.path.exists(correct_path))
print(os.listdir(correct_path))


Exists? True
['MOD13Q1.061__250m_16_days_VI_Quality_doy2003193000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2005193000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2003161000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_NDVI_doy2001241000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2004209000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2007225000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2003209000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2001177000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_NDVI_doy2007145000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2005177000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_NDVI_doy2002225000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_NDVI_doy2001209000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2005225000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_VI_Quality_doy2007161000000_aid0001.tif', 'MOD13Q1.061__250m_16_days_NDVI_doy200719300

In [13]:
from pathlib import Path
# Use the correct path that printed Exists? True
project_dir = Path("/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244")

# Find all NDVI TIFs
ndvi_paths = sorted(project_dir.glob("*NDVI*.tif"))

print(len(ndvi_paths))
ndvi_paths[:5]  # show first few

49


[PosixPath('/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244/MOD13Q1.061__250m_16_days_NDVI_doy2001145000000_aid0001.tif'),
 PosixPath('/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244/MOD13Q1.061__250m_16_days_NDVI_doy2001161000000_aid0001.tif'),
 PosixPath('/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244/MOD13Q1.061__250m_16_days_NDVI_doy2001177000000_aid0001.tif'),
 PosixPath('/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244/MOD13Q1.061__250m_16_days_NDVI_doy2001193000000_aid0001.tif'),
 PosixPath('/workspaces/data/gila_vegetation_data/gric_ndvi/MOD13Q1.061_2001137_to_2007244/MOD13Q1.061__250m_16_days_NDVI_doy2001209000000_aid0001.tif')]

### Repeating tasks in Python

Now you should have a few dozen files! For each file, you need to:

-   Load the file in using the `rioxarray` library
-   Get the date from the file name
-   Add the date as a dimension coordinate
-   Give your data variable a name

You don’t want to write out the code for each file! That’s a recipe for
**copy pasta** and errors. Luckily, Python has tools for doing similar
tasks repeatedly. In this case, you’ll use one called a `for` loop.

There’s some code below that uses a `for` loop in what is called an
**accumulation pattern** to process each file. That means that you will
save the results of your processing to a list each time you process the
files, and then merge all the arrays in the list.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ul>
<li>Look at the file names. How many characters from the end is the
date? <code>doy_start</code> and <code>doy_end</code> are used to
extract the day of the year (doy) from the file name. You will need to
count characters from the end and change the values to get the right
part of the file name. HINT: the index -1 in Python means the last
value, -2 second-to-last, and so on.</li>
<li>Replace any required variable names with your chosen variable
names</li>
</ul></div></div>

In [17]:
from pathlib import Path

doy_start = -25
doy_end = -18

# Loop through each NDVI image
ndvi_das = []
for ndvi_path in ndvi_paths:
    ndvi_path = Path(ndvi_path)
    #print(ndvi_path)
    # Get date from file name
    doy = ndvi_path.name[doy_start:doy_end]
    date = pd.to_datetime(doy, format='%Y%j')

    # Open dataset
    da = rxr.open_rasterio(ndvi_path, mask_and_scale=True).squeeze()

    # Add date dimension and clean up metadata
    
    da = da.assign_coords({'date': date})
    da = da.expand_dims({'date': 1})
    da.name = 'NDVI'

    # Prepare for concatenation
    ndvi_das.append(da)

print(ndvi_das)


[<xarray.DataArray 'NDVI' (date: 1, y: 203, x: 382)> Size: 310kB
array([[[0.8282    , 0.6146    , 0.3796    , ..., 0.1542    ,
         0.1542    , 0.1774    ],
        [0.52349997, 0.32029998, 0.46359998, ..., 0.145     ,
         0.145     , 0.145     ],
        [0.52349997, 0.38509998, 0.4567    , ..., 0.0962    ,
         0.058     , 0.0641    ],
        ...,
        [0.13759999, 0.13759999, 0.1283    , ..., 0.1489    ,
         0.1489    , 0.1568    ],
        [0.1299    , 0.1299    , 0.12379999, ..., 0.1831    ,
         0.1831    , 0.23169999],
        [0.1347    , 0.1309    , 0.1329    , ..., 0.15709999,
         0.2384    , 0.2452    ]]], shape=(1, 203, 382), dtype=float32)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3kB -112.3 -112.3 -112.3 ... -111.5 -111.5 -111.5
  * y            (y) float64 2kB 33.39 33.39 33.38 33.38 ... 32.97 32.97 32.97
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 8B 2001-05-25
Attributes:
    units:      

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Next, stack your arrays by date into a time series:</p>
<ol type="1">
<li>Modify the code to match your prior workflow steps and to use
descriptive variable names</li>
<li>Replace <code>coordinate_name</code> with the actual name of the
coordinate you want to build up.</li>
</ol></div></div>

In [18]:
# Combine NDVI images from all dates
da = xr.combine_by_coords(ndvi_das, coords=['date'])
da

/tmp/ipykernel_1555/3619952311.py:2: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  da = xr.combine_by_coords(ndvi_das, coords=['date'])
/tmp/ipykernel_1555/3619952311.py:2: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  da = xr.combine_by_coords(ndvi_das, coords=['date'])


<xarray.Dataset> Size: 15MB
Dimensions:      (date: 49, y: 203, x: 382)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3kB -112.3 -112.3 -112.3 ... -111.5 -111.5 -111.5
  * y            (y) float64 2kB 33.39 33.39 33.38 33.38 ... 32.97 32.97 32.97
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 392B 2001-05-25 2001-06-10 ... 2007-08-29
Data variables:
    NDVI         (date, y, x) float32 15MB 0.8282 0.6146 ... 0.2712 0.2602

In [ ]:
# Calculate the Difference between the times
ndvi_diff = (
    da.sel(date=slice('2004', '2007'))
    .mean('date')
    .NDVI

    - da.sel(date=slice('2001', '2004'))
    .mean('date')
    .NDVI
)

ndvi_diff

<xarray.DataArray 'NDVI' (y: 203, x: 382)> Size: 310kB
array([[-0.01323569,  0.01657507, -0.01090002, ...,  0.00726785,
         0.00726785,  0.00302139],
       [ 0.01914638,  0.01640359, -0.02488217, ...,  0.00558928,
         0.00558928,  0.0127786 ],
       [ 0.00573927,  0.01037496,  0.00883934, ...,  0.01500712,
         0.0038143 , -0.00166427],
       ...,
       [ 0.01912858,  0.01912858,  0.01547143, ...,  0.01068215,
         0.01068215,  0.01325715],
       [ 0.01512501,  0.01512501,  0.01928571, ...,  0.01450714,
         0.01450714,  0.00383213],
       [ 0.02069999,  0.02036427,  0.02151428, ...,  0.00607504,
         0.00087148,  0.00855   ]], shape=(203, 382), dtype=float32)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3kB -112.3 -112.3 -112.3 ... -111.5 -111.5 -111.5
  * y            (y) float64 2kB 33.39 33.39 33.38 33.38 ... 32.97 32.97 32.97
    spatial_ref  int64 8B 0

In [23]:
# Plot the difference

(ndvi_diff.hvplot(x='x', y='y', cmap='PiYG',
                 geo=True, title='Change in NDVI @ Gile River Community\n2004-2007 vs 2001-2004')

*

gric_gdf.hvplot(geo=True, fill_color=None, line_color='grey'))


:Overlay
   .Image.I    :Image   [x,y]   (NDVI)
   .Polygons.I :Polygons   [Longitude,Latitude]

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [11]:
%store var1 var2

Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!